In [260]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np 
from sklearn.preprocessing import MultiLabelBinarizer
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.compat import lzip

pd.set_option('display.max_columns', 500)
plt.rcParams["figure.figsize"] = (30, 10)

In [261]:
data = pd.read_csv('df.csv')
data

,aso,brand,capacity,color,condition,features,first_owner,fuel_type,horse_power,mileage,model,no_accidents,number_of_doors,origin_country,price,price_currency,transmission,type,url,version,year
0,Tak,Rolls-Royce,6 749 cm3,Czarny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,599 KM,49 000 km,Cullinan,Tak,5,NaN,1616999,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-cull...,Black Badge,2020
1,NaN,Rolls-Royce,6 750 cm3,Niebieski,Używane,NaN,NaN,Benzyna,335 KM,48 000 km,Silver Spur,NaN,5,NaN,298800,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1991
2,Tak,Rolls-Royce,6 592 cm3,Szary,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,632 KM,1 800 km,Wraith,Tak,2,Włochy,1950000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,2019
3,NaN,Rolls-Royce,5 379 cm3,Biały,Używane,"ABS,Immobilizer,Alufelgi,Gniazdo SD,Podgrzewan...",Tak,Benzyna,326 KM,86 400 km,Silver Seraph,NaN,4,NaN,197000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
4,NaN,Rolls-Royce,4 000 cm3,Czarny,Używane,NaN,Tak,Benzyna,130 KM,50 000 km,Wraith,NaN,4,Wielka Brytania,25000,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,1952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93265,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/W0VBD6EC8KG37603...,NaN,NaN
93266,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/WVGZZZ1TZHW11255...,NaN,NaN
93267,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/WVGZZZ1TZHW11255...,NaN,NaN
93268,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://otomotoklik.pl/oferta/TMBCE7NP8G708332...,NaN,NaN


In [262]:
data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage'])
data

,aso,brand,capacity,color,condition,features,first_owner,fuel_type,horse_power,mileage,model,no_accidents,number_of_doors,origin_country,price,price_currency,transmission,type,url,version,year
0,Tak,Rolls-Royce,6 749 cm3,Czarny,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Benzyna,599 KM,49 000 km,Cullinan,Tak,5,NaN,1616999,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-cull...,Black Badge,2020
2,Tak,Rolls-Royce,6 592 cm3,Szary,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,632 KM,1 800 km,Wraith,Tak,2,Włochy,1950000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-wrai...,NaN,2019
3,NaN,Rolls-Royce,5 379 cm3,Biały,Używane,"ABS,Immobilizer,Alufelgi,Gniazdo SD,Podgrzewan...",Tak,Benzyna,326 KM,86 400 km,Silver Seraph,NaN,4,NaN,197000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
5,NaN,Rolls-Royce,6 750 cm3,Biały,Używane,"Elektryczne szyby przednie,Klimatyzacja manual...",NaN,Benzyna,200 KM,650 000 km,Silver Shadow,NaN,4,NaN,59900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1975
6,Tak,Rolls-Royce,5 379 cm3,Zielony,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Benzyna,326 KM,58 650 km,Silver Seraph,NaN,4,Kanada,189900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/rolls-royce-silv...,NaN,1999
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93258,Tak,Volvo,1 969 cm3,Biały,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",Tak,Diesel,190 KM,175 000 km,XC 60,Tak,5,Polska,139000,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/volvo-xc-60-4x4-...,D4 AWD Momentum,2017
93259,Tak,Volvo,1 560 cm3,Inny kolor,Używane,"ABS,Elektryczne szyby przednie,Poduszka powiet...",NaN,Diesel,109 KM,249 000 km,V50,Tak,5,Belgia,14700,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/volvo-v50-lift-1...,NaN,2008
93260,Tak,Volvo,1 969 cm3,Złoty,Używane,"ABS,Elektrycznie ustawiane lusterka,Poduszka p...",Tak,Hybryda,320 KM,110 000 km,XC 90,Tak,5,Polska,223860,PLN,Manualna,NaN,https://www.otomoto.pl/oferta/volvo-xc-90-pole...,NaN,2016
93262,Tak,Volvo,1 969 cm3,Niebieski,Używane,"ABS,Elektrycznie ustawiane lusterka,Poduszka p...",Tak,Hybryda,303 KM,28 242 km,XC 90,Tak,5,Szwecja,357900,PLN,Automatyczna,NaN,https://www.otomoto.pl/oferta/volvo-xc-90-xc90...,T8 AWD Plug-In Hybrid Inscription 7os,2020


In [263]:
data.isna().sum()

aso                34164
brand                  0
capacity               0
color                  0
condition              0
features               0
first_owner        44623
fuel_type              0
horse_power            0
mileage                0
model                  0
no_accidents       26717
number_of_doors      312
origin_country     25854
price                  0
price_currency         0
transmission          21
type               71950
url                    0
version            29142
year                   0
dtype: int64

# Co do zrobienia z danymi? 
- aso na 0-1 X 
- OH brand X
- capacity na int X
- OH color X
- condition 0-1 X
- OH features X 
- first_owner bool X 
- fuel_type OH X 
- horse_power na int X 
- mileage  git na int X
- no_accidents X
- number_of_doors X 
- origin country OH 
- price - Y 
- transmission - if automatic 
- year 

In [264]:
def map_aso(row): 
    if row == 'Tak': 
        return 1 
    else: 
        return 0 

def map_capacity(row): 
    split_list = row.split(' ')
    try:  
        if len(split_list) == 2:
            return int(''.join(split_list[:1]))
        else: 
            return int(''.join(split_list[:2]))
            
    except ValueError: 
        return row
        
def map_condition(row): 
    if row == 'Używane':
        return 0
    else: 
        return 1 

def map_transmission(row): 
    if row == 'Automatyczna': 
        return 1 
    else :
        return 0 
        

In [265]:
data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage']).reset_index(drop=True)
data['features'] = data['features'].dropna().apply(lambda x: x.split(','))

lb = MultiLabelBinarizer()
features = lb.fit_transform(data['features'])
features = pd.DataFrame(features, columns=lb.classes_)

data = data.loc[data['fuel_type'].isin(['Benzyna', 'Diesel', 'Hybryda', 'Benzyna+LPG', 'Elektryczny'])]

df = pd.concat([data.drop(columns='features'), features], axis=1)
df = pd.concat([df.drop(columns='brand'), pd.get_dummies(df['brand'], drop_first=True)], axis=1)
df = pd.concat([df.drop(columns='fuel_type'), pd.get_dummies(df['fuel_type'], drop_first=True)], axis=1)

In [271]:
df['capacity'].isna().value_counts()

False    71939
True        42
Name: capacity, dtype: int64

In [267]:
# data = data.dropna(subset=['features', 'capacity', 'horse_power', 'mileage'])
# data['features'] = data['features'].dropna().apply(lambda x: x.split(','))

# lb = MultiLabelBinarizer()
# features = lb.fit_transform(data['features'])
# features = pd.DataFrame(features, columns=lb.classes_)

# data = data.loc[data['fuel_type'].isin(['Benzyna', 'Diesel', 'Hybryda', 'Benzyna+LPG', 'Elektryczny'])]

# df = pd.concat([data.drop(columns='features'), features], axis=1)
# df = pd.concat([df.drop(columns='brand'), pd.get_dummies(df['brand'], drop_first=True)], axis=1)
# df = pd.concat([df.drop(columns='fuel_type'), pd.get_dummies(df['fuel_type'], drop_first=True)], axis=1)

df['aso'] = df['aso'].apply(map_aso)
df['capacity'] = df['capacity'].apply(map_capacity)
df['horse_power'] = df['horse_power'].dropna().apply(map_capacity)
df['mileage'] = df['mileage'].dropna().apply(map_capacity)
df['no_accidents'] = df['no_accidents'].apply(map_aso)
df['first_owner'] = df['first_owner'].apply(map_aso)
df['condition'] = df['condition'].apply(map_condition)

# df = pd.concat([df.drop(columns='color'), pd.get_dummies(df['color'], drop_first=True)], axis=1)
df['transmission'] = df['transmission'].apply(map_transmission)
df = (df.rename(columns={'transmission': 'automatyczna', 'condition': 'new'})
        .drop(columns=['origin_country', 'model', 'type', 'url', 'version', 'price_currency', 'color'])
        .dropna()
        .astype(int)
        .reset_index(drop=True))


AttributeError: 'float' object has no attribute 'split'

In [ ]:
df

,aso,capacity,new,first_owner,horse_power,mileage,no_accidents,number_of_doors,price,automatyczna,year,ABS,ASR (kontrola trakcji),Alarm,Alufelgi,Asystent parkowania,Asystent pasa ruchu,Bluetooth,CD,Centralny zamek,Czujnik deszczu,Czujnik martwego pola,Czujnik zmierzchu,Czujniki parkowania przednie,Czujniki parkowania tylne,Dach panoramiczny,ESP (stabilizacja toru jazdy),Elektrochromatyczne lusterka boczne,Elektrochromatyczne lusterko wsteczne,Elektryczne szyby przednie,Elektryczne szyby tylne,Elektrycznie ustawiane fotele,Elektrycznie ustawiane lusterka,Gniazdo AUX,Gniazdo SD,Gniazdo USB,HUD (wyświetlacz przezierny),Hak,Immobilizer,Isofix,Kamera cofania,Klimatyzacja automatyczna,Klimatyzacja czterostrefowa,Klimatyzacja dwustrefowa,Klimatyzacja manualna,Komputer pokładowy,Kurtyny powietrzne,MP3,Nawigacja GPS,Odtwarzacz DVD,Ogranicznik prędkości,Ogrzewanie postojowe,Podgrzewana przednia szyba,Podgrzewane lusterka boczne,Podgrzewane przednie siedzenia,Podgrzewane tylne siedzenia,Poduszka powietrzna chroniąca kolana,Poduszka powietrzna kierowcy,Poduszka powietrzna pasażera,Poduszki boczne przednie,Poduszki boczne tylne,Przyciemniane szyby,Radio fabryczne,Radio niefabryczne,Regulowane zawieszenie,Relingi dachowe,System Start-Stop,Szyberdach,Tapicerka skórzana,Tapicerka welurowa,Tempomat,Tempomat aktywny,Tuner TV,Wielofunkcyjna kierownica,Wspomaganie kierownicy,Zmieniarka CD,features,Łopatki zmiany biegów,Światła LED,Światła Xenonowe,Światła do jazdy dziennej,Światła przeciwmgielne,Audi,Bentley,Chevrolet,Citroën,Dacia,Ferrari,Fiat,Kia,Lamborghini,Land Rover,Lexus,MINI,Maserati,Mazda,McLaren,Mercedes-Benz,Mitsubishi,Porsche,Renault,Rolls-Royce,Saab,Seat,Suzuki,Toyota,Volvo,Škoda,Benzyna+LPG,Diesel,Hybryda
0,1,6749,0,1,599,49000,1,5,1616999,1,2020,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,0,1,0,0,0,1,0,1,1,1,1,1,1,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
1,1,6592,0,0,632,1800,1,2,1950000,1,2019,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
2,0,5379,0,1,326,86400,0,4,197000,1,1999,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
3,0,6750,0,0,200,650000,0,4,59900,1,1975,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,1,0,1,1,1,1,1,0,0,0,0,0,1,0,0,1,1,0,0,1,1,0,1,1,1,0,0,0,1,1,0,1,1,0,0,0,1,0,1,0,1,0,1,0,0,0,1,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
4,1,5379,0,0,326,58650,0,4,189900,1,1999,1,1,1,1,0,1,1,0,1,1,0,1,1,1,0,1,1,1,1,1,1,1,1,0,1,1,0,1,1,1,1,0,1,0,1,1,1,1,0,1,0,0,1,1,0,1,1,1,1,1,0,1,0,1,0,0,0,1,0,0,1,0,1,1,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55176,0,998,1,0,67,10,1,5,53690,0,2021,1,1,1,1,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,1,1,0,0,0,0,1,0,0,0,0,1,0,1,0,0,1,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55177,0,998,1,0,67,10,1,5,53690,0,2021,1,1,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,1,1,0,1,0,0,0,1,1,1,0,0,0,0,0,1,0,0,0,1,1,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
55178,1,1396,0,1,90,21000,1,5,49900,0,2017,1,1,1,1,1,1,1,0,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,0,0,1,1,1,1,1,1,1

In [ ]:
df.dtypes.value_counts()

int64    111
dtype: int64

In [ ]:
Y = df['price']
X = df.loc[:, df.columns != 'price']
lm = sm.OLS(Y, X).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.830
Model:                            OLS   Adj. R-squared (uncentered):              0.830
Method:                 Least Squares   F-statistic:                              2687.
Date:                Sun, 16 Jan 2022   Prob (F-statistic):                        0.00
Time:                        23:08:05   Log-Likelihood:                     -6.8761e+05
No. Observations:               55181   AIC:                                  1.375e+06
Df Residuals:                   55081   BIC:                                  1.376e+06
Df Model:                         100                                                  
Covariance Type:            nonrobust                                                  
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------

In [ ]:
X2 = df.drop(columns=lb.classes_).drop(columns='price')
Y = df['price']
lm2 = sm.OLS(Y, X2).fit()
print(lm.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.830
Model:                            OLS   Adj. R-squared (uncentered):              0.830
Method:                 Least Squares   F-statistic:                              2687.
Date:                Sun, 16 Jan 2022   Prob (F-statistic):                        0.00
Time:                        23:08:05   Log-Likelihood:                     -6.8761e+05
No. Observations:               55181   AIC:                                  1.375e+06
Df Residuals:                   55081   BIC:                                  1.376e+06
Df Model:                         100                                                  
Covariance Type:            nonrobust                                                  
                                            coef    std err          t      P>|t|      [0.025      0.975]
--------------

In [ ]:
name = ["Lagrange multiplier statistic", "p-value", "f-value", "f p-value"]
test = sms.het_breuschpagan(lm.resid, lm.model.exog)
lzip(name, test)

[('Lagrange multiplier statistic', 11504.1154990286),
 ('p-value', 0.0),
 ('f-value', 145.07861378892568),
 ('f p-value', 0.0)]

In [ ]:
name = ["F statistic", "p-value"]
test = sms.het_goldfeldquandt(lm.resid, lm.model.exog)
lzip(name, test)

[('F statistic', 0.21472761453255299), ('p-value', 0.9999999999999999)]

In [ ]:
name = ["Jarque-Bera", "Chi^2 two-tail prob.", "Skew", "Kurtosis"]
test = sms.jarque_bera(lm.resid)
lzip(name, test)

[('Jarque-Bera', 46888339.97834025),
 ('Chi^2 two-tail prob.', 0.0),
 ('Skew', 3.920271568698204),
 ('Kurtosis', 145.58955106775971)]

In [ ]:
np.linalg.cond(lm.model.exog)

5.633034536005067e+38

In [ ]:
sms.linear_reset(lm)

/home/kuba1302/UW/ekonometria_projekt/venv/lib/python3.9/site-packages/statsmodels/stats/diagnostic.py:1081: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  aug = res.fittedvalues[:, None]


<class 'statsmodels.stats.contrast.ContrastResults'>
<Wald test (chi2): statistic=65440.44359676682, p-value=0.0, df_denom=2>